
# AutoEncoder
*train: 불러온 데이터     
*train_1: train 데이터 필요한 부분만 모아놓은 것        
*train_bin: train_1데이터를 이진수의 나열로 바꾼 데이터


In [ ]:
#csv 불러오기
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

from google.colab import drive
drive.mount('/content/drive')
train_name= '/content/drive/MyDrive/AI대회/데이터셋/AIclass/car_train.csv'
test_name= '/content/drive/MyDrive/AI대회/데이터셋/AIclass/car_test.csv'

train= pd.read_csv(train_name)
test= pd.read_csv(test_name)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#학습: 36만
train.shape

(360032, 5)

In [ ]:
#테스트: 8만
test.shape

(2000733, 5)

# [1]데이터 전처리



In [ ]:
train.head()

,Timestamp,Datetime,Arbitration_ID,DLC,Data
0,1597707573,2020-08-17 18:39,391,8,00 00 00 00 00 00 04 77
1,1597707573,2020-08-17 18:39,251,8,F0 03 A3 EE 00 F3 07 80
2,1597707573,2020-08-17 18:39,2B0,6,1D 00 00 07 83 AE
3,1597707573,2020-08-17 18:39,4F1,4,20 00 60 41
4,1597707573,2020-08-17 18:39,50C,8,40 0E 00 08 06 10 00 D2


In [ ]:
#데이터 필요없는 부분 자르기


#필요한 열만 뽑아내기
train_1 = train[['Arbitration_ID','DLC','Data']]

#쓸데없는 인덱스 제거

#train_1이 최종 trainset임**
train_1.head()

,Arbitration_ID,DLC,Data
0,391,8,00 00 00 00 00 00 04 77
1,251,8,F0 03 A3 EE 00 F3 07 80
2,2B0,6,1D 00 00 07 83 AE
3,4F1,4,20 00 60 41
4,50C,8,40 0E 00 08 06 10 00 D2


##1.flooding공격

In [ ]:
#전처리 부분


##1. Flooding 공격 피쳐 추출

###조건(1) 속도가 3600이상일때:frequent 변수로 학습 
train_1['freq']=train.groupby('Timestamp')['Timestamp'].transform('count')
train_1.head()

,Arbitration_ID,DLC,Data,freq
0,391,8,00 00 00 00 00 00 04 77,1042
1,251,8,F0 03 A3 EE 00 F3 07 80,1042
2,2B0,6,1D 00 00 07 83 AE,1042
3,4F1,4,20 00 60 41,1042
4,50C,8,40 0E 00 08 06 10 00 D2,1042


In [ ]:

###조건(2):  ID값이 0 이면서, 동시에 data00 이고. DLC 8일 때: is_Flooding열로 학습

def is_Flooding(df):

    if df['Arbitration_ID']=='0'and df['Data']=='00 00 00 00 00 00 00 00'and df['DLC']==8:
        return 1
    else :
        return 0
train_1['is_Flooding']=train_1.apply(is_Flooding,axis=1)

train_1.head()


,Arbitration_ID,DLC,Data,freq,is_Flooding
0,391,8,00 00 00 00 00 00 04 77,1042,0
1,251,8,F0 03 A3 EE 00 F3 07 80,1042,0
2,2B0,6,1D 00 00 07 83 AE,1042,0
3,4F1,4,20 00 60 41,1042,0
4,50C,8,40 0E 00 08 06 10 00 D2,1042,0


##2.Spoofing공격

In [ ]:
##2. Spoofing 공격 피쳐 추출
###조건(1): ID가 553이면서 데이터가 0000000201008000문자열을 가질 때
###조건(2): ID가 336이면서 28E2FF28250001 문자열을 가질 때

def is_Spoofing(df):
    if df['Arbitration_ID']=='553'and df['Data']=='00 00 00 02 01 00 80 00':
        return 1
    elif  df['Arbitration_ID']=='336'and df['Data']=='28 E2 FF 28 25 00 01':
        return 1
    else :
        return 0

train_1['is_Spoofing']=train_1.apply(is_Spoofing,axis=1)
train_1.head()
###조건(3): 1초당 약 2400개 패킷이 잡힐 때 : frequent 열로 학습


,Arbitration_ID,DLC,Data,freq,is_Flooding,is_Spoofing
0,391,8,00 00 00 00 00 00 04 77,1042,0,0
1,251,8,F0 03 A3 EE 00 F3 07 80,1042,0,0
2,2B0,6,1D 00 00 07 83 AE,1042,0,0
3,4F1,4,20 00 60 41,1042,0,0
4,50C,8,40 0E 00 08 06 10 00 D2,1042,0,0


##3.Replay공격

In [ ]:
##3. Replay 공격 피쳐 추출
###조건(1): 무조건 한번 이상은 출현되었던 패킷
###조건(2): 정상 패킷의 순서를 그대로 따름 

"""##==> 구현했으나, 오류 발생.

list_train=list(train['Arbitration_ID'])
train['replay_depth']=0
x=0#list_train에서 인덱스
y=0#list_check에서 인덱스
depth=0#궁극적인 output임.list_check에서 얼마나 내려갈 수 있는가?
list_uniqueID=[]#이전에 나왔던 id인가?
list_check=[]#이전에 나왔던 패킷의 흐름들(replay라면 동일할 확률높음)
depth_list=[]#중복패킷을 위해: 하나의x의 여러 depth중 가장 큰 수로 환원함. 
yList=[]#list_check에서의 인덱스 y의 후보들의 리스트(일시적으로 씀)


while list_train[x] != None : 
    
    if (list_train[x] in list_uniqueID)  :
        
        yList=[i for i, value in enumerate(list_check) if value ==list_train[x]]
        
        for y in yList:
            y=y+1
            if list_check[y] == None :#yList를 끝까지 다 봤을 때 while문 탈출
                break
            depth=depth+1
            if (list_check[y] not in list_train[x+1:x+31]) : #30개패킷 내에서 replay공격을 하는지 여부 
                depth_list.append(depth)#공격을 30개 내에서 안하면 당시의 depth를 저장하기
                #당연히 반복될 경우에는 for문 반복, 패킷이 없으면 depth 저장 후 다음 반복
        #
        train['replay_depth'][x]=max(depth_list)
        depth_list=[]
    else : 
        list_uniqueID.append(list_train[x])
        train['replay_depth'][x]=0
    #
    list_check.append(list_train[x])
    x=x+1
train['replay_depth']

"""

###조건(3):  1초당 약 2600~2700개 패킷이 잡힐 때  : freq 변수로 학습




"##==> 구현했으나, 오류 발생.\n\nlist_train=list(train['Arbitration_ID'])\ntrain['replay_depth']=0\nx=0#list_train에서 인덱스\ny=0#list_check에서 인덱스\ndepth=0#궁극적인 output임.list_check에서 얼마나 내려갈 수 있는가?\nlist_uniqueID=[]#이전에 나왔던 id인가?\nlist_check=[]#이전에 나왔던 패킷의 흐름들(replay라면 동일할 확률높음)\ndepth_list=[]#중복패킷을 위해: 하나의x의 여러 depth중 가장 큰 수로 환원함. \nyList=[]#list_check에서의 인덱스 y의 후보들의 리스트(일시적으로 씀)\n\n\nwhile list_train[x] != None : \n    \n    if (list_train[x] in list_uniqueID)  :\n        \n        yList=[i for i, value in enumerate(list_check) if value ==list_train[x]]\n        \n        for y in yList:\n            y=y+1\n            if list_check[y] == None :#yList를 끝까지 다 봤을 때 while문 탈출\n                break\n            depth=depth+1\n            if (list_check[y] not in list_train[x+1:x+31]) : #30개패킷 내에서 replay공격을 하는지 여부 \n                depth_list.append(depth)#공격을 30개 내에서 안하면 당시의 depth를 저장하기\n                #당연히 반복될 경우에는 for문 반복, 패킷이 없으면 depth 저장 후 다음 반복\n        #\n        train['replay_depth'][x]=max(d

##4. Fuzzing공격

In [ ]:
##4. Fuzzing 공격 피쳐 추출

###조건(1): DLC가 8이면서 고유한 data+Id값 일때
##데이터 빈도수 체크
train_1['freqData']=train.groupby('Data')['Data'].transform('count')

def is_Fuzzing(df):
    if df['DLC']==8:
      if df['freqData'] == 1:
        return 1
      return 0
    else :
        return 0
train_1['is_Fuzzing']=train_1.apply(is_Fuzzing,axis=1)

train_1

###조건(2):  1초당 약 2900~3100개 패킷이 잡힐 때 : freq 변수로 학습


,Arbitration_ID,DLC,Data,freq,is_Flooding,is_Spoofing,freqData,is_Fuzzing
0,391,8,00 00 00 00 00 00 04 77,1042,0,0,470,0
1,251,8,F0 03 A3 EE 00 F3 07 80,1042,0,0,3,0
2,2B0,6,1D 00 00 07 83 AE,1042,0,0,471,0
3,4F1,4,20 00 60 41,1042,0,0,245,0
4,50C,8,40 0E 00 08 06 10 00 D2,1042,0,0,39,0
...,...,...,...,...,...,...,...,...
360027,391,8,00 00 00 00 00 00 08 EB,503,0,0,470,0
360028,260,8,06 39 1A 30 FF D1 A1 63,503,0,0,1,1
360029,421,8,FE 07 00 FF E3 7F 00 9E,503,0,0,500,0
360030,130,8,94 8E F0 81 00 00 0B AC,503,0,0,1,1


#[2]이미지화 전처리



##1.이진화(16또는 10진수->2진수)


In [ ]:
##2진수로바꾸기
train_1['bin_ID'] = train['Arbitration_ID'].apply(lambda x: format(int(x,16),'04b'))
train_1['bin_DLC'] = train['DLC'].apply(lambda x: format(int(x), '04b'))
train_1['bin_freq'] = train_1['freq'].apply(lambda x: format(int(x), '04b'))
train_1['bin_freqData'] = train_1['freqData'].apply(lambda x: format(int(x), '04b'))

train_1['bin_Flooding'] = train_1['is_Flooding'].apply(lambda x: format(int(x), '04b'))##이 세가지 경우, 0이면 0000, 1이면 1111로 채울 수 있도록 수정필요. 
train_1['bin_Spoofing'] = train_1['is_Spoofing'].apply(lambda x: format(int(x), '04b'))
train_1['bin_Fuzzing'] = train_1['is_Fuzzing'].apply(lambda x: format(int(x), '04b'))

#Data따로 2진수로 바꾸는 과정: 공백열 제거 필요하기 때문. 
##Data 부분의 공백 열 제거하기
train['reData']=train.Data.apply(lambda x: x.replace(" ",""))

#받은 문자열에 대해 이진화해주기(16진수 문자열-> 2진수 문자열)
train_1['bin_Data'] = train.reData.apply(lambda x: format(int(x,16), '04b'))



##2.자릿수맞게 0으로 채워주기
이미지화이기때문에 각 위치에 특정 부분이 잘 자리할 수 있도록 함.


In [ ]:
train_1['bin_ID']=train_1.bin_ID.apply(lambda x: x.zfill(12))#ID는 12자리
train_1['bin_Data']=train_1.bin_Data.apply(lambda x: x.zfill(64))#Data는 64자리
train_1['bin_freq']=train_1.bin_freq.apply(lambda x: x.zfill(12))#freq는 12자리(4000(10)까지 가능)
train_1['bin_freqData']=train_1.bin_freqData.apply(lambda x: x.zfill(14))#freqData는 14자리(10000 까지 가능)


##3.train_bin: 2진수로만 이루어진 데이터 셋 준비 완료


In [ ]:
train_bin = train_1[['bin_ID','bin_DLC','bin_Data','bin_freq','bin_Flooding','bin_Spoofing','bin_Fuzzing','bin_freqData']]
train_bin

,bin_ID,bin_DLC,bin_Data,bin_freq,bin_Flooding,bin_Spoofing,bin_Fuzzing,bin_freqData
0,001110010001,1000,0000000000000000000000000000000000000000000000...,010000010010,0000,0000,0000,00000111010110
1,001001010001,1000,1111000000000011101000111110111000000000111100...,010000010010,0000,0000,0000,00000000000011
2,001010110000,0110,0000000000000000000111010000000000000000000001...,010000010010,0000,0000,0000,00000111010111
3,010011110001,0100,0000000000000000000000000000000000100000000000...,010000010010,0000,0000,0000,00000011110101
4,010100001100,1000,0100000000001110000000000000100000000110000100...,010000010010,0000,0000,0000,00000000100111
...,...,...,...,...,...,...,...,...
360027,001110010001,1000,0000000000000000000000000000000000000000000000...,000111110111,0000,0000,0000,00000111010110
360028,001001100000,1000,0000011000111001000110100011000011111111110100...,000111110111,0000,0000,0001,00000000000001
360029,010000100001,1000,1111111000000111000000001111111111100011011111...,000111110111,0000,0000,0000,00000111110100
360030,000100110000,1000,1001010010001110111100001000000100000000000000...,000111110111,0000,0000,0001,00000000000001


In [ ]:

train_1.index
train_1.columns

Index(['Arbitration_ID', 'DLC', 'Data', 'freq', 'is_Flooding', 'is_Spoofing',
       'freqData', 'is_Fuzzing', 'bin_ID', 'bin_DLC', 'bin_freq',
       'bin_freqData', 'bin_Flooding', 'bin_Spoofing', 'bin_Fuzzing',
       'bin_Data'],
      dtype='object')

In [ ]:
sttr=train_bin['bin_ID'][0]+train_bin['bin_DLC'][0]+train_bin['bin_Data'][0]+train_bin['bin_freq'][0]+train_bin['bin_Flooding'][0]+train_bin['bin_Spoofing'][0]+train_bin['bin_Fuzzing'][0]+train_bin['bin_freqData'][0]
sttr

'0011100100011000000000000000000000000000000000000000000000000000000001000111011101000001001000000000000000000111010110'

In [ ]:
type(sttr)

str

##4. 한 행마다 이미지화 하기
* 이미지화해서 저장하는 과정: 오류는 안나지만 저장 안됨. 

In [ ]:
### 아직 구현 하지 못함. 
##한 행마다 : (1) 각 열을 불러와서 연속적인 바이너리 열로 바꿔서 value값에 넣기-> (2)이미지화 과정 거쳐서 저장하기, ->  반복. 
from PIL import Image

def fix_missing3(df):
    for i in df.index: #0~360032
        value = "" #val 초기화
        for k in df.columns: #bin_ID, bin_Data...
            bin_val = df[k][i] #각 위치에 있는 바이너리 값을 들여옴, 
            value += bin_val #기존 값에 더하기
        #value = "0110100001100101011011000110110001101111"
        carr = np.array([(255,255,255), (0,0,0)], dtype='uint8')
        data = carr[np.array(map(int, list(value)))].reshape(-1, 8, 3)
        img = Image.fromarray(data, 'RGB')
        img.show()
        img.save('/content/drive/MyDrive/AI대회/out.png', 'PNG')





##5. 패딩
* 28*28이미지의 입력값이라는 전제하에 구현




In [ ]:
def pad(img,layers):
    #이미지는 정 사각형이어야함. 
    return [[0]*(len(img[0])+2*layers)]*layers    + \
           [[0]*layers+r+[0]*layers for r in img] + \
           [[0]*(len(img[0])+2*layers)]*layers


pad(train_bin,28)

#[3]모델 구성 및 학습
* 아래 오픈소스를 참고하여 작성함.  
(https://github.com/Seratna/TensorFlow-Convolutional-AutoEncoder/blob/master/convolutional_autoencoder.py)


##1-1. 오토인코더 모델

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf

model = models.Sequential()

#Dropout
model.add(Dropout(0.5)) #오류발생: 왜 오류가 나지?
#encoder
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D(pool_size=2,strides=2))
model.add(layers.Conv2D(64, (5,5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=2,strides=2))
#decoder
model.add(layers.Conv2DTranspose(64, (5, 5), activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(64, (5, 5), activation='relu', padding='same',strides=2))
model.add(layers.Conv2DTranspose(32, (13, 13), activation='relu', padding='same',strides=2))
model.add(layers.Conv2DTranspose(32, (5, 5), activation='relu', padding='same'))

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_20 (Conv2DT (None, 7, 7, 64)          102464    
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 14, 14, 64)        102464    
_________________________________________________________________
conv2d_transpose_22 (Conv2DT (None, 28, 28, 32)       

##2. 모델 학습

In [ ]:
#모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])

#Currently not used
callback_list = [EarlyStopping(monitor='val_accuracy', min_delta=0,
                               patience=2,
                               verbose=0, mode='auto')]


# 학습하기
hist = model.fit(Xtrain, Xtrain_n, batch_size = 16, epochs = EPOCHS, verbose = 0)
##Xtrain : train_1의 이미지화된 데이터
##Xtrain_n : train_1의 노이즈 첨가된 데이터



IndentationError: ignored